In [ ]:
from loader import retrieve_polo
from analyze_correlation import *

In [ ]:

# test_simularity()
# print('INDEX   TIMESTAMP            OPEN    CLOSE   HIGH    LOW     VOLUME      MOVE')

x = 200
y = 200
c = 300

In [ ]:
# segment_polo_ltc_30 = retrieve_polo('USDT_LTC', 30, '2017-01-01', '2020-09-15') # 요구된 봉 데이터를 Poloniex 거래소 API로 읽어와서 n x 6 크기의 numpy array로 만들어주는 함수
# segment_polo_ltc_5 = retrieve_polo('USDT_LTC', 5, '2017-01-01', '2020-09-15') # 요구된 봉 데이터를 Poloniex 거래소 API로 읽어와서 n x 6 크기의 numpy array로 만들어주는 함수

# segment_polo_btc_30 = retrieve_polo('USDT_BTC', 30, '2017-01-01', '2020-09-15') # 요구된 봉 데이터를 Poloniex 거래소 API로 읽어와서 n x 6 크기의 numpy array로 만들어주는 함수
# segment_polo_btc_5 = retrieve_polo('USDT_BTC', 5, '2017-01-01', '2020-09-15') # 요구된 봉 데이터를 Poloniex 거래소 API로 읽어와서 n x 6 크기의 numpy array로 만들어주는 함수

segment_polo_ltc_30 = load_poloniex_candles_from_csv('USDT_LTC', 30)
segment_polo_ltc_5 = load_poloniex_candles_from_csv('USDT_LTC', 5)

segment_polo_btc_30 = load_poloniex_candles_from_csv('USDT_BTC', 30)
segment_polo_btc_5 = load_poloniex_candles_from_csv('USDT_BTC', 5)

cluster_info = {}
cluster_info['segment'] = segment_polo_btc_5[-1000:,:]
cluster_info['clustering'] = get_clusters(segment_polo_btc_5, x, y, c)

instance = 'polo_btc_5_' + str(c)

import pickle

with open('cluster_info.' + instance + '.pickle.staging', 'wb') as f:
    pickle.dump(cluster_info, f)


In [ ]:
import pickle

def load_cluster_info(instance):
    with open('cluster_info.' + instance + '.pickle', 'rb') as f:
        cluster_info = pickle.load(f)
        print(len(cluster_info['segment']))
        print(cluster_info['clustering'])
    return cluster_info

cluster_db = {}

cluster_db[instance] = load_cluster_info(instance + '_100000')

In [ ]:
find_common_permutations(segment, clustering, c)

In [ ]:
def find_prio_rise(segment, clustering, c, max_loss_ratio, min_profit_ratio):
    global snapshots_count

    n = segment.shape[0]
    r = n - x

    ltc = Series(segment)

    match_count = np.full(c, 0, dtype=np.int32)
    member_count = np.full(c, 0, dtype=np.int32)

    for i in range(r-y):
        j = i + x
        cluster = clustering.labels_[i]
        member_count[cluster] += 1
        if segment[j,1] / min(segment[j:j+y,4]) < max_loss_ratio and max(segment[j:j+y,3]) / segment[j,1] > min_profit_ratio:
            # if snapshots_count < max_snapshots:
            #     ax = fig.add_subplot(gs_all[0, snapshots_count])
            #     ltc.draw(ax, j)
            #     snapshots_count += 1
            match_count[cluster] += 1
    a = np.transpose(np.array([match_count / member_count, match_count, member_count, np.arange(c, dtype=np.int32)]))
    return a[a[:,0].argsort()]

cluster_info = cluster_db[instance]

segment = cluster_info['segment']
clustering = cluster_info['clustering']

# prio_rise = find_prio_rise(segment, clustering, c, 1.03, 1.05)
prio_rise = find_prio_rise(segment, clustering, c, 1.005, 1.02)
for i in range(c):
    if prio_rise[i,0] > 0.5:
        print(prio_rise[i,0], prio_rise[i,1], prio_rise[i,2], prio_rise[i,3])


In [ ]:
snapshots_count = 0
fig = plt.figure(figsize=(20 * max_snapshots, 7 * coins_count))
gs_all = gridspec.GridSpec(coins_count, max_snapshots, figure=fig)

n = segment.shape[0]
r = n - x

ltc = Series(segment)

last = None

for i in range(r-y):
    j = i + x
    if clustering.labels_[i] == 37 and (last is None or last < i - 50):
        last = i
        if snapshots_count < max_snapshots:
            ax = fig.add_subplot(gs_all[0, snapshots_count])
            ltc.draw(ax, j)
            snapshots_count += 1

print('Rendering...')
fig.savefig("analyze_correlation.png", bbox_inches='tight', dpi=100)
plt.close(fig)
print('done')
